In [2]:
import numpy as np
import random
import qsharp

from qiskit import QuantumCircuit

from qsharp.estimator import EstimatorParams, ErrorBudgetPartition, LogicalCounts
from qsharp.interop.qiskit import estimate, ResourceEstimatorBackend
from collections import defaultdict
import qiskit
from qiskit.compiler import transpile
import matplotlib.pyplot as plt

import itertools
import time
import concurrent

import os
from tqdm.notebook import tqdm

from multiprocessing import Pool, Manager, Queue, Value, Array
import multiprocessing 

from IPython.display import clear_output

### Generate a Numpy Array Consisting of Logical Counts of the Circuits and the Corresponding Optimized Error Budgets

In [ ]:
def find_optimized_budgets(total_budget, num_iterations, counts):
    parameters = EstimatorParams()
    default_parameters = EstimatorParams()
    default_parameters.error_budget = total_budget

    default_result = counts.estimate(default_parameters)
    logical_counts = default_result['logicalCounts']

    default_physicalqubits = default_result['physicalCounts']['physicalQubits']
    default_runtime = default_result['physicalCounts']['runtime']

    running_metric = None
    loss = False

    running_optimal_parameters = {}
    logical_cache = []
    t_cache = []
    rotation_cache = []

    for i in range(num_iterations):
        parameters.error_budget = ErrorBudgetPartition()
        logical_budget_random = np.random.uniform(0, 1)
        t_budget_random = np.random.uniform(0, 1)
        rotation_budget_random = np.random.uniform(0, 1)
        budget_sum = logical_budget_random + t_budget_random + rotation_budget_random

        logical_budget = (logical_budget_random / budget_sum) * total_budget
        t_budget = (t_budget_random / budget_sum) * total_budget
        rotation_budget = (rotation_budget_random / budget_sum) * total_budget

        parameters.error_budget.logical = logical_budget
        parameters.error_budget.t_states = t_budget
        parameters.error_budget.rotations = rotation_budget
        
        result = counts.estimate(params=parameters)
        default_result = counts.estimate()

        physicalqubits = result['physicalCounts']['physicalQubits']
        runtime = result['physicalCounts']['runtime']
        
        default_metric = default_runtime * default_physicalqubits
        current_metric = runtime * physicalqubits
        
        if running_metric == None:
            running_metric = current_metric
            logical_cache = logical_budget
            t_cache = t_budget
            rotation_cache = rotation_budget
        
        if current_metric < running_metric:
            running_metric = current_metric
            logical_cache = logical_budget
            t_cache = t_budget
            rotation_cache = rotation_budget

    if running_metric > default_metric:
        loss = True

    running_optimal_parameters['logical_budget'] = logical_cache
    running_optimal_parameters['t_budget'] = t_cache
    running_optimal_parameters['rotation_budget'] = rotation_cache

    return list(running_optimal_parameters.values()), loss, running_metric, default_metric


def generate_data(total_error_budget, counts):
    qasm_files = [os.path.join(root, file) for root, _, files in os.walk("MQTBench") for file in files if file.endswith(".qasm")]
    qasm_files = sorted(qasm_files)

    results = []

    for file in qasm_files:
        with open(file, 'r') as f:
            qasm = f.read()   
            qc = QuantumCircuit.from_qasm_str(qasm)
        estimation = estimate(qc)
        counts = estimation['logicalCounts']
        if counts['rotationCount'] == 0:
            continue
        counts = LogicalCounts(counts)
        combinations, loss, running_metric, default_metric = find_optimized_budgets(total_error_budget, 1000, counts)
        specific_data = [int(counts['numQubits']),int(counts['tCount']),int(counts['rotationCount']),int(counts['rotationDepth']),int(counts['cczCount']),int(counts['ccixCount']),int(counts['measurementCount'])]
        specific_data += combinations
        results.append(specific_data)

    return results

### Save the Data

In [ ]:
results = generate_data(0.1, 1000)

result_array = np.asarray(results)
print(np.shape(result_array))
print(result_array.dtype)
np.save('demonstration_data_ten_percent.npy', result_array)